In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("./pre_processed_train.csv",index_col="id")
test = pd.read_csv("./pre_processed_test.csv",index_col="id")

X=train.copy()
X_test=test.copy()


y=X.label
y_test=X_test.label

X=X.drop(['label'],axis=1)
X_test=X_test.drop(['label'],axis=1)

train.head()

,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,...,Tokenized_Segment_1601,Tokenized_Segment_1602,Tokenized_Segment_1603,Tokenized_Segment_1604,Tokenized_Segment_1605,Tokenized_Segment_1606,Tokenized_Segment_1607,Tokenized_Segment_1608,Tokenized_Segment_1609,Tokenized_Segment_1610
id,,,,,,,,,,,,,,,,,,,,,
33181,1,0.8,1521614353,5,1.806818,116,72.64,15.04,89.26,1.00,...,0,0,0,0,0,0,0,0,0,0
2606,0,1.0,1527009817,4,9.429737,109,79.08,76.85,56.75,98.18,...,0,0,0,0,0,0,0,0,0,0
38816,1,0.8,1535935605,2,7.769821,167,33.80,76.38,86.24,25.77,...,0,0,0,0,0,0,0,0,0,0
239,1,0.6,1516429555,0,2.667798,273,2.98,15.25,95.42,79.26,...,0,0,0,0,0,0,0,0,0,0
1421,1,0.8,1539809005,24,7.554238,89,32.22,28.71,84.01,1.00,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# evaluate a given model using cross-validation
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
def evaluate_model(model, X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=42)
    model.fit(X_train, y_train)
    
    y_hat = model.predict(X_valid)
    
    return f1_score(y_valid,y_hat,average="binary")


In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
# optuna to optimize
import optuna
from sklearn.model_selection import cross_val_score

In [ ]:
def gbm_objective(trial):
    
    n_estimators = trial.suggest_int("n_estimators", 300, 5000)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    min_samples_split=trial.suggest_int("min_samples_split", 100, 1000,step=10)
    model = GradientBoostingClassifier(n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         subsample=subsample, 
                        min_samples_split=min_samples_split,
                        n_iter_no_change=50,
                                        validation_fraction=0.2,
                         )  
    return evaluate_model(model, X, y)

study_gbm = optuna.create_study(study_name='gbm_optimze',direction='maximize')
study_gbm.optimize(gbm_objective, n_trials=100)

[I 2021-08-29 15:52:10,229] A new study created in memory with name: gbm_optimze
[I 2021-08-29 15:52:32,361] Trial 0 finished with value: 0.775244299674267 and parameters: {'n_estimators': 1542, 'max_depth': 9, 'learning_rate': 0.08840330346147143, 'subsample': 0.9, 'min_samples_split': 720}. Best is trial 0 with value: 0.775244299674267.
[I 2021-08-29 15:55:27,279] Trial 1 finished with value: 0.7448680351906158 and parameters: {'n_estimators': 3175, 'max_depth': 6, 'learning_rate': 4.745562556004935e-05, 'subsample': 0.6, 'min_samples_split': 550}. Best is trial 0 with value: 0.775244299674267.
[I 2021-08-29 15:57:33,578] Trial 2 finished with value: 0.7717041800643086 and parameters: {'n_estimators': 2724, 'max_depth': 1, 'learning_rate': 0.010704928251883118, 'subsample': 0.9, 'min_samples_split': 240}. Best is trial 0 with value: 0.775244299674267.
[I 2021-08-29 16:01:35,082] Trial 3 finished with value: 0.7732463295269169 and parameters: {'n_estimators': 3267, 'max_depth': 5, 'le